In [ ]:
!pip install langchain-community
!pip install chromadb
!pip install sentence_transformers
!pip install pypdf
!pip install tiktoken
!pip install chromadb

In [2]:
from langchain.vectorstores import Chroma
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import pipeline
import torch
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import SentenceTransformerEmbeddings

In [4]:
loader = DirectoryLoader("/content/ipc-data", glob="./*.pdf", loader_cls=PyPDFLoader)
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=200)

texts = text_splitter.split_documents(documents)

In [5]:
texts

15148

In [7]:

embeddings = SentenceTransformerEmbeddings(model_name="multi-qa-mpnet-base-dot-v1")

subset_texts = texts[:30]

persist_directory = "/content/db_directory"
db = Chroma.from_documents(subset_texts, embeddings, persist_directory=persist_directory)

db.persist()

db = Chroma(persist_directory=persist_directory, embedding_function=embeddings)

print(f"Number of chunks in subset: {len(subset_texts)}")
print("Database created and persisted with the subset of text chunks.")


Number of chunks in subset: 30
Database created and persisted with the subset of text chunks.


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 0.4. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  warn_deprecated(


In [33]:
checkpoint = "MBZUAI/LaMini-Flan-T5-783M"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
base_model = AutoModelForSeq2SeqLM.from_pretrained(
    checkpoint,
    device_map="auto",
    torch_dtype=torch.float32
)

In [35]:
pipe = pipeline(
    'text2text-generation',
    model = base_model,
    tokenizer = tokenizer,
    max_length = 512,
    do_sample = True,
    temperature = 0.3,
    top_p= 0.95
)


In [36]:
local_llm = HuggingFacePipeline(pipeline=pipe)
qa_chain = RetrievalQA.from_chain_type(
    llm=local_llm,
    chain_type='stuff',
    retriever=db.as_retriever(search_type="similarity", search_kwargs={"k": 2}),
    return_source_documents=True,
)


In [56]:
input_query = str(input("Enter your query:"))

llm_response = qa_chain({"query": input_query})

print(llm_response['result'])


Enter your query:what is ipc
IPC stands for Indian Penal Code.


In [12]:

tokenizer.save_pretrained("tokenizer_model")
base_model.save_pretrained("base_model")


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2525: UserWarning: Attempting to save a model with offloaded modules. Ensure that unallocated cpu memory exceeds the `shard_size` (5GB default)
  warnings.warn(
